# Image_to_music
Welcome to image to music.
You can consult our code or just skip to the end to try in out ! Provide an image that you put in the folder and put its name as the file variable, and then run all the cells

In [1]:
import math
from musicpy import *
from midi2audio import FluidSynth
import random

mode_list = [
    'dorian',
    'phrygian',
    'minor',
    'major',
    'lydian',
    'mixolydian',
    'locrian'
]

color_chord_map = {
    '#FF0000': 'C',
    '#FFA500': 'G',
    '#FFFF00': 'D',
    '#008000': 'A',
    '#ADD8E6': 'E',
    '#87CEEB': 'B',
    '#0000FF': 'F',
    '#EE82EE': 'Db',
    '#800080': 'Ab',
    '#B0C4DE': 'Eb',
    '#5F9EA0': 'Bb',
    '#8B0000': 'F',
}


def color_to_chord_map(color, color_map):
    approx_color = closest_color_from_hex(color, color_map)
    return color_map.get(approx_color, None)


def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(*rgb)

def color_distance(rgb1, rgb2):
    return math.sqrt(sum((c1 - c2) ** 2 for c1, c2 in zip(rgb1, rgb2)))

def closest_color_from_hex(color, color_map):
    rbg_color = hex_to_rgb(color)
    closest_colors = sorted(color_map.keys(), key=lambda color: color_distance(rbg_color, hex_to_rgb(color)))
    return closest_colors[0]

def closest_color(rgb, color_map):
    closest_colors = sorted(color_map.keys(), key=lambda color: color_distance(rgb, hex_to_rgb(color)))
    return closest_colors[0]

def freq_to_note_name(freq):
    # This is a simplification, in reality, you would need to account for tuning and the exact frequencies of notes
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    return notes[round(math.log2(freq / 440.0) * 12) % 12]

def wich_instrument():
    instrument = 0
    for color in color_chord_map.values():
        instrument += color
    instrument = instrument % 128
    return int(instrument)

def divide_by_100(array):
    divided_array = []
    for sublist in array:
        divided_sublist = [value / 100 for value in sublist]
        divided_array.append(divided_sublist)
    return divided_array

def round_to_nearest(value):
    multiples = [3.1, 6.25, 25, 12.5, 33, 50, 66, 75, 100]
    nearest = min(multiples, key=lambda x: abs(x - value))
    return nearest

def round_values(array):
    rounded_array = []
    for sublist in array:
        rounded_sublist = [round_to_nearest(value) for value in sublist]
        rounded_array.append(rounded_sublist)
    return rounded_array

def calculate_percentages(array):
    percentages_array = []
    for sublist in array:
        suite_en_cours = [sublist[0]]
        percentages = []
        for i in range(1, len(sublist)):
            if sublist[i] == suite_en_cours[-1]:
                suite_en_cours.append(sublist[i])
            else:
                suite_length = len(suite_en_cours)
                percentage = (suite_length / len(sublist)) * 100
                percentages.append(percentage)
                suite_en_cours = [sublist[i]]
        # Calcul du pourcentage pour la dernière suite en cours
        suite_length = len(suite_en_cours)
        percentage = (suite_length / len(sublist)) * 100
        percentages.append(percentage)

        total_percentage = sum(percentages)
        rounded_percentages = [round(percentage) for percentage in percentages]
        diff = 100 - sum(rounded_percentages)
        # Distribuer la différence d'arrondi en ajoutant ou en soustrayant 1 au premier ou au dernier élément
        if diff > 0:
            rounded_percentages[-1] += diff
        elif diff < 0:
            rounded_percentages[0] += diff
        percentages_array.append(rounded_percentages)

    return divide_by_100(round_values(percentages_array))


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default


In [2]:
def compress_list(arr):
    compressed_lists = []
    for lst in arr:
        compressed_list = [lst[0]]
        for i in range(1, len(lst)):
            if lst[i] != lst[i - 1]:
                compressed_list.append(lst[i])
        compressed_lists.append(compressed_list)
    return compressed_lists


In [3]:
def closest_fraction_power_of_two(decimal_number, round_to_one=True):
    """
    Finds the closest decimal number to the input that is a fraction of a power of two (1/2**n).

    Args:
    - decimal_number (float): The decimal number to find the closest fraction of a power of two for.
    - round_to_one (bool): If the function should ever return 1
    Returns:
    - float: The closest fraction of a power of two to the input decimal number.
    """
    # Calculate the power of two using logarithms to find the closest fraction
    if decimal_number > 0:
        n = -math.log(decimal_number, 2)
        # Round to the nearest whole number to find the closest power of two fraction
        n_rounded = round(n)
        # Convert back to the fraction of a power of two
        closest_fraction = 1 / (2 ** n_rounded)
    else:
        closest_fraction = 0  # For non-positive inputs, return 0
    if (closest_fraction == 1.0) and not round_to_one:
        return 1/2
    return closest_fraction

In [4]:
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans

def load_and_cluster(path, n_clusters, display=False):
    image = Image.open(path)
    x, y = image.size

    h = 150 / float(y)
    redim = min(x, h)
    new_h = int(y * redim)
    image = image.resize((x, new_h), Image.LANCZOS)

    x, y = image.size

    if image.mode != 'RGB':
        image = image.convert('RGB')

    pixels = list(image.getdata())
    pixels_list = [pixels[i * x:(i + 1) * x] for i in range(y)]
    image_array = np.array(image)
    pixels = image_array.reshape(-1, 3)

    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(pixels)
    labels = kmeans.labels_

    for i in range(y):
        for j in range(x):
            pixel_index = i * x + j
            pixels_list[i][j] = labels[pixel_index]
    centers = [rgb_to_hex(i) for i in kmeans.cluster_centers_.astype(int)]
    return centers, pixels_list

In [5]:
def generate_pattern_number(n, seed=None):
    if seed is not None:
        random.seed(seed)
    pattern_number = 0
    for _ in range(n):
        digit = random.randint(1, 6)
        pattern_number = pattern_number * 10 + digit
    return pattern_number

In [6]:
def generate_gaussian_between(b1, b2, mean, std_dev, seed=None):
    if seed is not None:
        np.random.seed(seed)

    while True:
        number = np.random.normal(mean, std_dev)
        if b1 <= number <= b2:
            return number

In [7]:
def weighted_mode_selection(mode_list, mean, std_dev, seed=None):
    if seed is not None:
        np.random.seed(seed)

    b1, b2 = 0, len(mode_list) - 1
    while True:
        index = int(np.round(np.random.normal(mean, std_dev)))
        if b1 <= index <= b2:
            return mode_list[index]


In [8]:

def generate_chords(color, color_map, rythmic_pattern, mode=None):
    chord_choice = color_to_chord_map(color, color_map)
    if mode is None:
        mode = weighted_mode_selection(mode_list, 3, 1)
    pattern_number = generate_pattern_number(1)

    # print(f"Chord:{chord_choice}\nMode:{mode}")
    # print(f"Pattern number:{pattern_number}\n")

    chords = S(f'{chord_choice} {mode}') % (pattern_number, 1/4, rythmic_pattern, len(rythmic_pattern))
    return chords

In [9]:
def image_to_music(main_colors, pixels, outputfile="output_song.mid"):
    chords = []
    percentages = calculate_percentages(pixels)
    colors = compress_list(pixels)

    nb_lines = len(colors)
    for i, line in enumerate(zip(percentages, colors)):
        for color in line[1]:
            selected_mode = weighted_mode_selection(mode_list=mode_list,
                                            mean=3,
                                            std_dev=1)
            hex_color = main_colors[color]
            chords += generate_chords(hex_color,
                                      color_map=color_chord_map,
                                      mode=selected_mode,
                                      rythmic_pattern=line[0])

    print(f"Writing to file: {outputfile}")
    mp.write(chords, bpm=100, name=outputfile)
    return outputfile


# Produce result

In [11]:
main_colors, pixels_list = load_and_cluster(file, 3)
print("Image loaded")
image_to_music(main_colors, pixels_list, outputfile=file.replace('png', 'mid'))
fs = FluidSynth()
fs.midi_to_audio(file.replace('png', 'mid'), file.replace('png', 'wav'))

Image loaded
i: 0/150
i: 1/150
i: 2/150
i: 3/150
i: 4/150
i: 5/150
i: 6/150
i: 7/150
i: 8/150
i: 9/150
i: 10/150
i: 11/150
i: 12/150
i: 13/150
i: 14/150
i: 15/150
i: 16/150
i: 17/150
i: 18/150
i: 19/150
i: 20/150
i: 21/150
i: 22/150
i: 23/150
i: 24/150
i: 25/150
i: 26/150
i: 27/150
i: 28/150
i: 29/150
i: 30/150
i: 31/150
i: 32/150
i: 33/150
i: 34/150
i: 35/150
i: 36/150
i: 37/150
i: 38/150
i: 39/150
i: 40/150
i: 41/150
i: 42/150
i: 43/150
i: 44/150
i: 45/150
i: 46/150
i: 47/150
i: 48/150
i: 49/150
i: 50/150
i: 51/150
i: 52/150
i: 53/150
i: 54/150
i: 55/150
i: 56/150
i: 57/150
i: 58/150
i: 59/150
i: 60/150
i: 61/150
i: 62/150
i: 63/150
i: 64/150
i: 65/150
i: 66/150
i: 67/150
i: 68/150
i: 69/150
i: 70/150
i: 71/150
i: 72/150
i: 73/150
i: 74/150
i: 75/150
1/2 done
i: 76/150
i: 77/150
i: 78/150
i: 79/150
i: 80/150
i: 81/150
i: 82/150
i: 83/150
i: 84/150
i: 85/150
i: 86/150
i: 87/150
i: 88/150
i: 89/150
i: 90/150
i: 91/150
i: 92/150
i: 93/150
i: 94/150
i: 95/150
i: 96/150
i: 97/150
i: 98/15

fluidsynth: error: fluid_is_soundfont(): fopen() failed: 'File does not exist.'
Parameter '/home/ionan/.fluidsynth/default_sound_font.sf2' not a SoundFont or MIDI file or error occurred identifying it.


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'Flag_of_France.wav'..
